In [ ]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation 
from p_drought_indices.analysis.DeepLearning.pipeline_convlstm import training_lstm
import pickle
import os
import matplotlib.pyplot as plt
from p_drought_indices.functions.function_clns import load_config, prepare, CNN_split, interpolate_prepare
import numpy as np
from p_drought_indices.analysis.DeepLearning.dataset import CustomDataset
from torch.utils.data import DataLoader

CONFIG_PATH = "../config.yaml"
sub_precp, ds = data_preparation(CONFIG_PATH)
sub_precp = sub_precp.to_dataset()
data, target = interpolate_prepare(sub_precp, ds)
train_split = 0.8
training_lstm(CONFIG_PATH, data, target, train_split = train_split)

In [ ]:
def data_CHECK(data_array:xr.DataArray):
    # Check shape and size
    print("Shape:", data_array.shape)
    print("Size:", data_array.size)
    
    # Check coordinates and dimensions
    print("Dimensions:", data_array.dims)
    print("Coordinates:", data_array.coords)
    
    # Check data type and values
    print("Data Type:", data_array.dtype)
    print("Data Values:", data_array.values)
    
    # Check for missing values
    print("Missing Values:", data_array.isnull().any())
    
    # Check attributes
    print("Attributes:", data_array.attrs)

In [3]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation, get_dataloader, main
from p_drought_indices.analysis.DeepLearning.pipeline_convlstm import training_lstm
import pickle
import os
import matplotlib.pyplot as plt
from p_drought_indices.functions.function_clns import load_config, prepare, CNN_split, interpolate_prepare, subsetting_pipeline, get_lat_lon_window
import numpy as np
from p_drought_indices.analysis.DeepLearning.dataset import CustomDataset
from torch.utils.data import DataLoader
import xarray as xr


CONFIG_PATH = "../config.yaml"
config = load_config(CONFIG_PATH)

c:\Users\Riccardo\anaconda3\envs\gis2_py39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
sub_precp, ds =  data_preparation(CONFIG_PATH, ndvi_dataset="smoothed_ndvi_1_old.nc")
x_df = sub_precp.to_dataframe()
x_df_1 = x_df.swaplevel(1,2)
for col in ["spatial_ref","crs"]:
    if col in x_df_1:
        x_df_1.drop(columns={col}, inplace=True)
x_df_2 = x_df_1.dropna(subset={"tp"})
x_df_3 = x_df_2.sort_values(["lat", "lon","time"],ascending=False)
data_x_unstack = x_df_3.unstack(["lat","lon"])
#x_unstack = data_x_unstack.to_numpy()
num_samples, num_nodes = data_x_unstack.shape
x_unstack = np.expand_dims(data_x_unstack, axis=-1)
print("The features have dimensions:", x_unstack.shape)

In [ ]:
y_df = ds.to_dataframe()
for col in ["spatial_ref","crs"]:
    if col in y_df:
        y_df.drop(columns={col}, inplace=True)
y_df = y_df.dropna(subset={"ndvi"})
y_df = y_df.sort_values(["lat", "lon","time"],ascending=False)
y_df = y_df.reset_index().set_index(["time","lon","lat"])
y_df = y_df[y_df.index.isin(x_df_3.index)]
data_y_unstack = y_df.unstack(["lat","lon"])
y_unstack = data_y_unstack.to_numpy()
y_unstack = np.expand_dims(y_unstack, axis=-1)
print("The instance have dimensions:", y_unstack.shape)

### changes with features
st_df = x_df_3.reset_index()[["lon","lat"]].drop_duplicates()

In [ ]:
for i in [x_df, x_df_1, x_df_2, x_df_3, data_x_unstack]:
    print(len(i))

In [ ]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import generate_adj_dist
adj_dist = generate_adj_dist(st_df)
seq_length_x = seq_length_y = 12
y_start = 1
x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))

with open(os.path.join(config["DEFAULT"]["data"], "graph_net/big_adj_dist.pkl"), 'wb') as f:
        pickle.dump(adj_dist, f, protocol=2)

In [ ]:
n_sub_precp, n_ds =  data_preparation(CONFIG_PATH, ndvi_dataset="smoothed_ndvi_1.nc")
n_x_df = n_sub_precp.to_dataframe()
n_x_df_1 = n_x_df.swaplevel(1,2)
n_x_df_2 = n_x_df_1.dropna(subset={"tp"}).drop(columns={"spatial_ref","crs"})
n_x_df_3 = n_x_df_2.sort_values(["lat", "lon","time"],ascending=False)
n_data_x_unstack = n_x_df_3.unstack(["lat","lon"])
#x_unstack = data_x_unstack.to_numpy()
n_num_samples, n_num_nodes = n_data_x_unstack.shape
n_x_unstack = np.expand_dims(n_data_x_unstack, axis=-1)
print("The features have dimensions:", n_x_unstack.shape)

In [ ]:
for i in [n_x_df, n_x_df_1, n_x_df_2, n_x_df_3, n_data_x_unstack]:
    print(len(i))

In [ ]:
n_y_df = n_ds.to_dataframe()
n_y_df = n_y_df.dropna(subset={"ndvi"}).drop(columns={"spatial_ref","crs"})
n_y_df = n_y_df.sort_values(["lat", "lon","time"],ascending=False)
n_y_df = n_y_df.reset_index().set_index(["time","lon","lat"])
n_y_df = n_y_df[n_y_df.index.isin(n_x_df_3.index)]
n_data_y_unstack = n_y_df.unstack(["lat","lon"])
n_y_unstack = n_data_y_unstack.to_numpy()
n_y_unstack = np.expand_dims(n_y_unstack, axis=-1)
print("The instance have dimensions:", n_y_unstack.shape)

### changes with features
n_st_df = n_x_df_3.reset_index()[["lon","lat"]].drop_duplicates()

In [ ]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import generate_adj_dist
adj_dist = generate_adj_dist(n_st_df)
with open(os.path.join(config["DEFAULT"]["data"], "graph_net/n_adj_dist.pkl"), 'wb') as f:
        pickle.dump(adj_dist, f, protocol=2)

In [ ]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import load_dataset, MetricsRecorder

output_dir = os.path.join(config["DEFAULT"]["data"],  "graph_net")
seq_length_x = seq_length_y = 12
y_start = 1
x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))

# Predict the next one hour
y_offsets = np.sort(np.arange(y_start, (seq_length_y + 1), 1))
x, y = [], []
min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive

for t in range(min_t, max_t):  # t is the index of the last observation.
    x.append(x_unstack[t + x_offsets, ...])
    y.append(y_unstack[t + y_offsets, ...])
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)
print("x shape: ", x.shape, ", y shape: ", y.shape)
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
x_test, y_test = x[-num_test:], y[-num_test:]
for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join(output_dir, f"{cat}.npz"),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )
batch_size = config["GWNET"]["batch_size"]
dataloader = load_dataset(output_dir, batch_size, batch_size, batch_size)

In [ ]:
import torch 
import time
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import save_figures, trainer, load_adj
device = "cpu" #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seq_length=12
nhid=32
in_dim =1
adjtype = "doubletransition"
learning_rate=0.001
dropout=0.3
weight_decay=0.0001
gcn_bool = "store_true"
addaptadj = "store_true"
print_every = 50
epochs = 50

adj_path = os.path.join(config["DEFAULT"]["data"], "graph_net/big_adj_dist.pkl")
adj_mx = load_adj(adj_path,  adjtype)
scaler = dataloader['scaler']
supports = [torch.tensor(i).to(device) for i in adj_mx]
metrics_recorder = MetricsRecorder()
adjinit = supports[0]
engine = trainer(scaler, in_dim, seq_length, num_nodes, nhid, dropout,
                     learning_rate, weight_decay, device, supports, gcn_bool, addaptadj,
                     adjinit)
print("start training...",flush=True)
his_loss =[]
val_time = []
train_time = []
for i in range(1,epochs+1):
    train_loss = []
    train_mape = []
    train_rmse = []
    t1 = time.time()
    dataloader['train_loader'].shuffle()
    for iter, (x, y) in enumerate(dataloader['train_loader'].get_iterator()):
        trainx = torch.Tensor(x).to(device)
        trainx= trainx.transpose(1, 3)
        trainy = torch.Tensor(y).to(device)
        trainy = trainy.transpose(1, 3)
        metrics = engine.train(trainx, trainy[:,0,:,:])
        train_loss.append(metrics[0])
        train_mape.append(metrics[1])
        train_rmse.append(metrics[2])
        if iter % print_every == 0 :
            log = 'Iter: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}'
            print(log.format(iter, train_loss[-1], train_mape[-1], train_rmse[-1]),flush=True)
    t2 = time.time()
    train_time.append(t2-t1)
    #validation
    valid_loss = []
    valid_mape = []
    valid_rmse = []
    s1 = time.time()
    for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)
        testy = torch.Tensor(y).to(device)
        testy = testy.transpose(1, 3)
        metrics = engine.eval(testx, testy[:,0,:,:])
        valid_loss.append(metrics[0])
        valid_mape.append(metrics[1])
        valid_rmse.append(metrics[2])
    
    s2 = time.time()
    log = 'Epoch: {:03d}, Inference Time: {:.4f} secs'
    print(log.format(i,(s2-s1)))
    val_time.append(s2-s1)
    mtrain_loss = np.mean(train_loss)
    mtrain_mape = np.mean(train_mape)
    mtrain_rmse = np.mean(train_rmse)
    metrics_recorder.add_train_metrics(mtrain_mape, mtrain_rmse, mtrain_loss)
    mvalid_loss = np.mean(valid_loss)
    mvalid_mape = np.mean(valid_mape)
    mvalid_rmse = np.mean(valid_rmse)
    his_loss.append(mvalid_loss)
    metrics_recorder.add_val_metrics(mvalid_mape, mvalid_rmse, mvalid_loss)
    save_figures(config=config, epoch=i, train_loss=metrics_recorder.train_loss, 
                train_mape=metrics_recorder.train_mape, train_rmse=metrics_recorder.train_rmse, 
                test_loss=metrics_recorder.val_loss, test_rmse=metrics_recorder.val_loss, 
                test_mape=metrics_recorder.val_mape)
    log = 'Epoch: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}, Valid Loss: {:.4f}, Valid MAPE: {:.4f}, Valid RMSE: {:.4f}, Training Time: {:.4f}/epoch'
    print(log.format(i, mtrain_loss, mtrain_mape, mtrain_rmse, mvalid_loss, mvalid_mape, mvalid_rmse, (t2 - t1)),flush=True)
    #torch.save(engine.model.state_dict(), args.save+"_epoch_"+str(i)+"_"+str(round(mvalid_loss,2))+".pth")
print("Average Training Time: {:.4f} secs/epoch".format(np.mean(train_time)))
print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))